In [1]:
import pandas as pd
import pandasql as ps

In [2]:
demand = pd.read_csv('demand_test.csv')
charge = pd.read_csv('charge_test.csv')

C:\Users\AM\miniforge3\lib\site-packages\IPython\core\interactiveshell.py:3155: DtypeWarning: Columns (9,10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
query1 = '''
select count(*)
from
    (
    select * from demand
    where LOAN_ISSUED = 1) d
    left join charge c on d.DEMAND_ID = c.DEMAND_ID
    where CHARGE_SUMMA < 0

'''

In [4]:
# десять платежей с отрицательной суммой
clients_with_negative_CHARGE_SUMMA = ps.sqldf(query1)
clients_with_negative_CHARGE_SUMMA

,count(*)
0,10


In [5]:
query2 = '''
select d.DEMAND_ID, SAVE_STAMP, CHARGE_DATE, LOAN_AMOUNT, CHARGE_SUMMA
from
    (
    select * from demand
    where LOAN_ISSUED = 1) d
    left join charge c on d.DEMAND_ID = c.DEMAND_ID
    where CHARGE_DATE < SAVE_STAMP

'''

In [6]:
# 1392 случая, когда заявка сохранена после первого платежа и иногда даже не одного
clients_with_negative_CHARGE_SUMMA = ps.sqldf(query2)
clients_with_negative_CHARGE_SUMMA

,DEMAND_ID,SAVE_STAMP,CHARGE_DATE,LOAN_AMOUNT,CHARGE_SUMMA
0,56493,2019-05-07 00:00:00,2019-05-06 00:00:00,13000,390.0
1,56558,2019-05-07 00:00:00,2019-05-06 00:00:00,10000,10150.0
2,56609,2019-05-07 00:00:00,2019-05-06 00:00:00,10900,1900.0
3,24376,2019-05-01 00:00:00,2019-04-30 00:00:00,9000,395.0
4,24376,2019-05-01 00:00:00,2019-04-30 00:00:00,9000,9135.0
...,...,...,...,...,...
1387,655890,2019-08-26 00:00:00,2019-08-25 00:00:00,15000,150.0
1388,655890,2019-08-26 00:00:00,2019-08-25 00:00:00,15000,150.0
1389,655890,2019-08-26 00:00:00,2019-08-25 00:00:00,15000,1050.0
1390,655890,2019-08-26 00:00:00,2019-08-25 00:00:00,15000,15000.0


#### 2 варианта

1. Исключить платежи до сохранения заявки:

In [7]:
query3 = '''
with base as 
    (
    select
        d.DEMAND_ID id,
        date(SAVE_STAMP, 'start of month') loan_date,
        LOAN_AMOUNT loan_amount,
        date(CHARGE_DATE, 'start of month') charge_date,
        CHARGE_SUMMA charge_amount,
        round((julianday(date(CHARGE_DATE, 'start of month')) - julianday(date(SAVE_STAMP, 'start of month'))) / 30) period
    from
        (
        select * from demand
        where LOAN_ISSUED = 1
        ) d
    left join charge c
    on d.DEMAND_ID = c.DEMAND_ID
    where CHARGE_SUMMA > 0 and CHARGE_DATE >= SAVE_STAMP
    )    
        

select
    cp.coghort,
    cp.period,
    borrowers,
    sum(payments) over (partition by cp.coghort order by cp.period
                rows between unbounded preceding and current row) / loans_amount * 100 payments_percentage
    
from
(
select
    loan_date coghort,
    period,
    count(id) borrowers,
    sum(charge_amount) payments
from base
group by
    coghort,
    period
) cp
left join
(
select
    loan_date coghort,
    sum(loan_amount) loans_amount
from (select distinct id, loan_date, loan_amount from base)
group by coghort
) ls

on ls.coghort = cp.coghort
'''

In [8]:
cohorts_1 = ps.sqldf(query3)

cohorts_1.head(60)

,coghort,period,borrowers,payments_percentage
0,2019-05-01,0.0,18863,38.024277
1,2019-05-01,1.0,24353,95.485641
2,2019-05-01,2.0,11042,117.836328
3,2019-05-01,3.0,9244,134.208233
4,2019-05-01,4.0,5106,144.688504
5,2019-05-01,5.0,2694,150.439482
6,2019-05-01,6.0,1543,153.869488
7,2019-05-01,7.0,782,155.329746
8,2019-05-01,8.0,455,156.074738
9,2019-05-01,9.0,423,157.031445


2. Отнять пару дней от даты сохранения заявки, на случай если вдруг эти записи должны принадлежать к апрельским когортам. 

In [9]:
query4 = '''
with base as 
    ( 
    select
        id,
        loan_date,
        loan_amount,
        charge_date,
        charge_amount,
        round((julianday(charge_date) - julianday(loan_date)) / 30) period
    from (
            select
                d.DEMAND_ID id,
                case when CHARGE_DATE < SAVE_STAMP then date(SAVE_STAMP, '-2 days', 'start of month')
                else date(SAVE_STAMP, 'start of month')
                end as loan_date,
                LOAN_AMOUNT loan_amount,
                date(CHARGE_DATE, 'start of month') charge_date,
                CHARGE_SUMMA charge_amount
            from
                (
                select * from demand
                where LOAN_ISSUED = 1
                ) d
            left join charge c
            on d.DEMAND_ID = c.DEMAND_ID
            where CHARGE_SUMMA > 0
    )    
        )

select
    cp.coghort,
    cp.period,
    borrowers,
    sum(payments) over (partition by cp.coghort order by cp.period
                rows between unbounded preceding and current row) / loans_amount * 100 payments_percentage
    
from
(
select
    loan_date coghort,
    period,
    count(id) borrowers,
    sum(charge_amount) payments
from base
group by
    coghort,
    period
) cp
left join
(
select
    loan_date coghort,
    sum(loan_amount) loans_amount
from (select distinct id, loan_date, loan_amount from base)
group by coghort
) ls

on ls.coghort = cp.coghort
'''

In [10]:
cohorts_2 = ps.sqldf(query4)

cohorts_2.head(59)

,coghort,period,borrowers,payments_percentage
0,2019-04-01,0.0,9,23.654551
1,2019-04-01,1.0,8,36.559594
2,2019-05-01,0.0,19201,38.283984
3,2019-05-01,1.0,24353,95.636561
4,2019-05-01,2.0,11042,117.944934
5,2019-05-01,3.0,9244,134.285843
6,2019-05-01,4.0,5106,144.746273
7,2019-05-01,5.0,2694,150.486363
8,2019-05-01,6.0,1543,153.909875
9,2019-05-01,7.0,782,155.367369
